In [73]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [74]:
import warnings
warnings.filterwarnings("ignore")

In [75]:
from ipywidgets import IntProgress
from IPython.display import display

In [76]:
from var_table import VarTable

In [77]:
from cost_calculator_factory import CostCalculatorFactory
from multibinary_converter import MultiBinaryConverter
from multidiscrete_converter import MultiDiscreteConverter
from fitness_calculator import FitnessCalculator

In [78]:
from genetic_multibinary_space_config import GeneticMultiBinarySpaceConfig
from genetic_multidiscrete_space_config import GeneticMultiDiscreteSpaceConfig
from genetic_config import GeneticConfig
from genetic_solver import GeneticSolver

In [79]:
from stable_baselines3 import PPO
from stable_baselines3 import A2C
from stable_baselines3.common.env_checker import check_env

In [80]:
from rl_multibinary_config import RlMultiBinaryConfig
from rl_multidiscrete_config import RlMultiDiscreteConfig
from rl_graphic import RlGraphic
from rl_env import RlEnv
from rl_solver import RlSolver

In [81]:
from model_builder import ModelBuilder
from solve_action import SolveAction

In [82]:
from cost_calculator_factory import CostCalculatorFactory

In [83]:
M = 10 ** 6
dtype = np.int64
policy = 'MlpPolicy'

In [84]:
var_table = VarTable()

In [ ]:
class ExperimentForGroup:
    def __init__(self, group_name, experiments_count):
        self.group_name = group_name
        self.experiments_count = experiments_count
        self.binary_genetic_fitnesses = []
        self.discrete_genetic_fitnesses = []
        self.binary_rl_a2c_fitnessses = []
        self.binary_rl_ppo_fitnessess = []
        self.discrete_rl_a2c_fitnesses = []
        self.discrete_rl_ppo_fitnesses = []

    def make(self):
        progress = IntProgress(min=0, max=self.experiments_count, value=0, description=self.group_name, bar_style='success')
        display(progress)
        for experiment_number in range(1, self.experiments_count + 1):
            self.make_experiment(experiment_number, progress)
        dataset = {
            'binary genetic': self.binary_genetic_fitnesses,
            'discrete genetic': self.discrete_genetic_fitnesses,
            'binary rl a2c': self.binary_rl_a2c_fitnessses,
            'binary rl ppo': self.binary_rl_ppo_fitnessess,
            'discrete rl a2c': self.discrete_rl_a2c_fitnesses,
            'discrete rl ppo': self.discrete_rl_ppo_fitnesses
        }
        self.violinplot(dataset)

    def violinplot(self, dataset):
        df = pd.DataFrame(dataset)
        df_melted = pd.melt(df)
        title = 'Statistic'
        sns.violinplot(x='variable', y='value', data=df_melted, width=0.5, dodge=False, hue='variable').set(title=title)
        plt.xticks(rotation=75)
        plt.xlabel('')
        plt.ylabel('fitness')

    def make_experiment(self, experiment_number, progress):
        nested_directory = var_table.get_nested_directory(self.group_name, experiment_number)
        T_file_path = '{}/T.csv'.format(nested_directory)
        D_file_path = '{}/D.csv'.format(nested_directory)
        C_file_path = '{}/C.csv'.format(nested_directory)
        E_file_path = '{}/E.csv'.format(nested_directory)
        T_df = pd.read_csv(T_file_path, index_col=False, header=None)
        D_df = pd.read_csv(D_file_path, index_col=False, header=None)
        C_df = pd.read_csv(C_file_path, index_col=False, header=None)
        E_df = pd.read_csv(E_file_path, index_col=False, header=None)
        T = T_df.to_numpy()
        D = D_df.to_numpy()
        C = C_df.to_numpy()
        E = E_df.to_numpy()
        cost_calculator_factory = CostCalculatorFactory(T, D, C, E)
        m = np.shape(T)[1]
        k = var_table.generate_k(self.group_name)

        binary_genetic_solver = self.create_binary_genetic_solver(m, k, cost_calculator_factory)
        discrete_genetic_solver = self.create_discrete_genetic_solver(m, k, cost_calculator_factory)
        binary_rl_a2c_solver = self.create_binary_rl_solver(m, k, cost_calculator_factory, A2C)
        binary_rl_ppo_solver = self.create_binary_rl_solver(m, k, cost_calculator_factory, PPO)
        discrete_rl_a2c_solver = self.create_discrete_rl_solver(m, k, cost_calculator_factory, A2C)
        discrete_rl_ppo_solver = self.create_discrete_rl_solver(m, k, cost_calculator_factory, PPO)
        
        binary_solution, binary_solution_fitness, binary_solution_idx = binary_genetic_solver.solve()
        discrete_solution, discrete_solution_fitness, discrete_solution_idx = discrete_genetic_solver.solve()
        binary_rl_a2c_fitness, binary_rl_a2c_solution = binary_rl_a2c_solver.solve()
        binary_rl_ppo_fitness, binary_rl_ppo_solution = binary_rl_ppo_solver.solve()
        discrete_rl_a2c_fitness, discrete_rl_a2c_solution = discrete_rl_a2c_solver.solve()
        discrete_rl_ppo_fitness, discrete_rl_ppo_solution = discrete_rl_ppo_solver.solve()

        self.binary_genetic_fitnesses.append(binary_solution_fitness)
        self.discrete_genetic_fitnesses.append(discrete_solution_fitness)
        self.binary_rl_a2c_fitnessses.append(binary_rl_a2c_fitness)
        self.binary_rl_ppo_fitnessess.append(binary_rl_ppo_fitness)
        self.discrete_rl_a2c_fitnesses.append(discrete_rl_a2c_fitness)
        self.discrete_rl_ppo_fitnesses.append(discrete_rl_ppo_fitness)

        progress.value += 1
        

    def create_binary_genetic_solver(self, m, k, cost_calculator_factory):
        converter = MultiBinaryConverter(m, k)
        fitness_calculator = FitnessCalculator(cost_calculator_factory, converter)
        space_config = GeneticMultiBinarySpaceConfig(m, k)
        config = GeneticConfig(space_config)
        description = 'binary genetic'
        progress = IntProgress(min=0, max=config.num_generations, value=0, description=description, bar_style='success')
        solver = GeneticSolver(config, fitness_calculator, progress)
        return solver

    def create_discrete_genetic_solver(self, m, k, cost_calculator_factory):
        converter = MultiDiscreteConverter(m, k)
        fitness_calculator = FitnessCalculator(cost_calculator_factory, converter)
        space_config = GeneticMultiDiscreteSpaceConfig(m, k)
        config = GeneticConfig(space_config)
        description = 'discrete genetic'
        progress = IntProgress(min=0, max=config.num_generations, value=0, description=description, bar_style='success')
        solver = GeneticSolver(config, fitness_calculator, progress)
        return solver
    
    def create_binary_rl_solver(self, m, k, cost_calculator_factory, algorithm):
        total_timesteps = 100
        config = RlMultiBinaryConfig(m, k, dtype)
        converter = MultiBinaryConverter(m, k)
        fitness_calcultor = FitnessCalculator(cost_calculator_factory, converter)
        progress = IntProgress(min=0, max=total_timesteps, value=0, description='multibinary', bar_style='success')
        graphic = RlGraphic('multibinary')
        env = RlEnv(config, fitness_calcultor, progress, graphic)
        check_env(env)
        model = algorithm(policy, env)
        solver = RlSolver(model, total_timesteps)
        return solver
    
    def create_discrete_rl_solver(self, m, k, cost_calculator_factory, algorithm):
        total_timesteps = 100
        config = RlMultiDiscreteConfig(m, k, dtype)
        converter = MultiDiscreteConverter(m, k)
        fitness_calculator = FitnessCalculator(cost_calculator_factory, converter)
        progress = IntProgress(min=0, max=total_timesteps, value=0, description='multidiscrete', bar_style='success')
        graphic = RlGraphic('multidiscrete')
        env = RlEnv(config, fitness_calculator, progress, graphic)
        check_env(env)
        model = algorithm(policy, env)
        solver = RlSolver(model, total_timesteps)
        return solver


In [86]:
group_name = 'G0'
experiments_count = 100

In [87]:
experiment_for_group = ExperimentForGroup(group_name, experiments_count)

In [ ]:
experiment_for_group.make()

IntProgress(value=0, bar_style='success', description='progress')